In [1]:
import copy
import numpy as np

from numpy.random import shuffle
import matplotlib.pyplot as plt
from tqdm import tqdm

import funcs as f
import utils as utl

In [2]:
def show(G):
    
    tmp = 'X | '
    for v in V:
        tmp += ' ' + v + ' '
    
    print(tmp)
    print('')
    
    for row in range(len(V)):
        
        tmp = V[row] + ' | '
        for col in range(Vsize):
            
            if G[row][col] == np.inf:
                tmp +=  ' i '
            else:
                w = int(G[row][col])
                if w == 0:
                    tmp +=  " - "
                else:
                    if w < 10 : tmp +=  ' ' + str(w) + ' '
                    if 10 <= w < 100 : tmp +=  ' ' + str(w)

            
        print(tmp)

---

![title](test_graph.png)

## Graph init

In [3]:
# |V| = |{S,E}| + |T|*|K| = 2 + 4*2 = 10
Vsize = 10
Esize = 22

#     0    1    2    3    4    5    6    7    8    9
V = ['S', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'E']

In [4]:
g = np.zeros((Vsize,Vsize), dtype=np.float32)
betta = 20
# S
g[0][1] = np.inf
g[0][5] = np.inf

# a
g[1][2] = 10 #q_t(k)
g[1][5] = betta

# b
g[2][1] = np.inf
g[2][3] = 5  #q_t(k)
g[2][6] = betta 

#c
g[3][2] = np.inf
g[3][4] = 10  #q_t(k)
g[3][7] = betta 

#d
g[4][3] = np.inf
g[4][9] = 8  #q_t(k)
g[4][8] = betta 


#e
g[5][6] = 6  #q_t(k)
g[5][1] = betta 

#f
g[6][5] = np.inf
g[6][7] = 8  #q_t(k)
g[6][2] = betta 

#g
g[7][6] = np.inf
g[7][8] = 3  #q_t(k)
g[7][3] = betta 

#h
g[8][7] = np.inf
g[8][9] = 5  #q_t(k)
g[8][4] = betta 

In [5]:
show(g)

X |  S  a  b  c  d  e  f  g  h  E 

S |  -  i  -  -  -  i  -  -  -  - 
a |  -  -  10 -  -  20 -  -  -  - 
b |  -  i  -  5  -  -  20 -  -  - 
c |  -  -  i  -  10 -  -  20 -  - 
d |  -  -  -  i  -  -  -  -  20 8 
e |  -  20 -  -  -  -  6  -  -  - 
f |  -  -  20 -  -  i  -  8  -  - 
g |  -  -  -  20 -  -  i  -  3  - 
h |  -  -  -  -  20 -  -  i  -  5 
E |  -  -  -  -  -  -  -  -  -  - 


In [6]:
g[g > 0].size # Esize

24

---

In [7]:
#@njit
def iteration(G, F, V):
    
    Prev = np.full(Vsize, -1) # Prev node
    Q = [0] # Queue
    cursor = 0 # instead of popping elements of of the Q

    while cursor < Vsize:

        v_curr = Q[cursor]
        for v_ind, capacity in enumerate( G[v_curr] ):
            if F[v_curr, v_ind] < capacity and Prev[v_ind] == -1 and v_ind != v_curr:
                Q.append(v_ind)
                Prev[v_ind] = v_curr

        cursor += 1
        if cursor >= len(Q) : break

    # path reconstruction
    v_curr = Vsize - 1
    path = [v_curr]
    while True:
        v_prev = Prev[v_curr]
        v_curr = v_prev
        if v_curr == -1 : break
        path.append(v_curr)

    path = path[::-1]
    
    
    
    # path bottleneck
    bottleneck = np.inf # float('inf') doesn't work with numba
    for i in range(len(path) - 1):
        v_curr = path[i]
        v_next = path[i+1]

        local_w = G[v_curr, v_next] - F_[v_curr, v_next]
        if local_w < bottleneck: 
            bottleneck = local_w
    if bottleneck == np.inf : return F, True
    
    print("path: ", path, " | bottleneck: ", bottleneck)
    
    # path flow update
    for i in range(len(path) - 1):
        v_curr = path[i]
        v_next = path[i+1]

        F[v_curr, v_next] += bottleneck
        
    
    #print('')
    #print('----- New iteration -----')
    #show_path(path, V)
    #print('Bottleneck is: ', bottleneck)
    
    return F, False

In [8]:
Vsize = g.shape[0]
F_ = np.zeros((Vsize, Vsize)) # Current flow

In [9]:
show(g)

X |  S  a  b  c  d  e  f  g  h  E 

S |  -  i  -  -  -  i  -  -  -  - 
a |  -  -  10 -  -  20 -  -  -  - 
b |  -  i  -  5  -  -  20 -  -  - 
c |  -  -  i  -  10 -  -  20 -  - 
d |  -  -  -  i  -  -  -  -  20 8 
e |  -  20 -  -  -  -  6  -  -  - 
f |  -  -  20 -  -  i  -  8  -  - 
g |  -  -  -  20 -  -  i  -  3  - 
h |  -  -  -  -  20 -  -  i  -  5 
E |  -  -  -  -  -  -  -  -  -  - 


In [10]:
while True:
        F_, end = iteration(g, F_, V)
        if end : break

path:  [0, 1, 2, 3, 4, 9]  | bottleneck:  5.0
path:  [0, 5, 6, 7, 8, 9]  | bottleneck:  3.0
path:  [0, 5, 6, 7, 3, 4, 9]  | bottleneck:  3.0
path:  [0, 1, 2, 6, 7, 3, 4, 8, 9]  | bottleneck:  2.0


In [11]:
show(F_)
print('MaxFlow is: ', sum(F_[:, -1]))

X |  S  a  b  c  d  e  f  g  h  E 

S |  -  7  -  -  -  6  -  -  -  - 
a |  -  -  7  -  -  -  -  -  -  - 
b |  -  -  -  5  -  -  2  -  -  - 
c |  -  -  -  -  10 -  -  -  -  - 
d |  -  -  -  -  -  -  -  -  2  8 
e |  -  -  -  -  -  -  6  -  -  - 
f |  -  -  -  -  -  -  -  8  -  - 
g |  -  -  -  5  -  -  -  -  3  - 
h |  -  -  -  -  -  -  -  -  -  5 
E |  -  -  -  -  -  -  -  -  -  - 
MaxFlow is:  13.0


---

## MinCut

In [12]:
# BFS
Prev = np.full(Vsize, -1) # Prev node
Q = [9] # Queue
cursor = 0 # instead of popping elements of of the Q

while True:

    v_curr = Q[cursor]
    for v_ind, capacity in enumerate( g[v_curr] ):
        if 0 < F_[v_curr][v_ind] < capacity and Prev[v_ind] == -1 and v_ind != v_curr:
            Q.append(v_ind)
            Prev[v_ind] = v_curr
    
    cursor += 1
    if cursor >= len(Q) : break

In [13]:
# S a b c d x y z g E
# 0 1 2 3 4 5 6 7 8 9
Q

[9]

In [14]:
res = []
for v in Q:
    res.append(V[v])
res

['E']